# PLease this time, before Anyone make any changes, please let me know, 
# Since Mam Has given us one more chance, I don't want to miss it
# thanks

## Import Statements

In [ ]:
# used to parse the files
import xml.etree.ElementTree as xmltree

# for searching the folders
from os.path import isdir
from os.path import isfile
from os import mkdir
from os import listdir

# File Processing Functions

In [ ]:
# this will directly dump the word_tag pairs from give file in the their corresponding word_tag files
# in sentence format

# so the format is 
# every line of dest file will be a sentence
# we will only consider word tags this time.
# and sentences will be stored as word_tag pairs

def parseFile_mk(source,dest) :
    
    # xmlfile is the file to be handled
    xmlfile = open (source, "r",encoding= "utf-8")
    # we get the text into a variable
    xmlfiletext = xmlfile.read();
    #close xmlfile
    xmlfile.close()
    
    # make am xml tree from that text
    tree = xmltree.fromstring(xmlfiletext)
    
    # make list of sentences
    slist = tree.iter('s')
    
    #open dest file with **overwriting** mode
    dfile = open (dest[:-4]+"_sentence.txt", "w+", encoding = "utf-8")
    
    
    # put the data in the sfile
    for sentence in slist :
        # declare a line variable to store the sentence
        line = ""
        # iterate over every variable in sentence
        for elem in sentence :
            #make sure we only process words for a sentence
            if elem.tag != 'w' :
                continue 
            # fetch pos tag of this elem
            pos = elem.get('c5')
            # if the child doesnot contains c5 then it is of no use for us
            if pos is None :
                continue
                
            # upper case all pos tags    
            pos = pos.upper()   
            
            # some where the authors were confused and they assigned two tags
            # in such scenarios we are deciding to go with first tag and ignore the second
#             pos = pos[:3]
            
            # if I am here it  means pos has c5 tag available so we need to find the text of it.
            txt = elem.text
            if txt is None : 
                continue
            txt = txt.strip()
            # append in the sentence this word
            line =   line +      txt + "_"+ pos        +" "  # extra space added between terms 
                                                             # to just make more clear
        # for word loop finshes
        # write the sentence to the file
        dfile.write(line+"\n")	
    # for sentence loop finished 
    #close file
    dfile.close()

In [ ]:
# function
# this will fetch all the files and generate corresponding files
# this takes source folder and 
def makeSentences_mk(source, dest) :

    #check if the source is a file
    if isfile(source):
        # if yes, then process the file 
        # if destination is not there , the function will create it, and if it is there, it will overrite it.
        parseFile_mk(source,dest)
    else:
        # so the source is a folder, we need to make corresponding folder in the dest 
        #check if dest has the folder
        if not isdir(dest):
            mkdir(dest)
        
        # now enumerate all the items in the source directory    
        for item in listdir(source):
            makeSentences_mk(source+'/'+item,dest+'/'+item)



# Please Uncomment & Run below Cell to Process Files, 
## I do processed my files so I have commented this to avoid repeats

In [ ]:
# this segment is used to make the files process all togethor in their own files

source = "given_data"
dest = "processed_data"
makeSentences_mk(source, dest)

# Combine Function to Combine all the files

In [ ]:
# Collector function, this will dump all sentences into the destination file
def dump_mk(source,dest):

    sfile = open(source, "r", encoding= "utf-8")
    dfile = open(dest, "a", encoding= "utf-8")

    # read entire sfile line by line, yes this is the key
    # Possible Error Point : in our Previous Implementation I was reading It all at same time, this was problem ,
        # whole file became a single sentence
    # and then dump that  into the dfile as itis.
    for line in sfile :
        dfile.write(line + "\n")

    sfile.close()
    dfile.close()

# this iterates and finds all the files and stores them into a single file 
# using dumping function
def combine_mk(source, dest) :

    #check if the source is a file
    if isfile(source):
        # if yes, then process the file 
        # if destination is not there , the function will create it, and if it is there, it will overrite it.
        dump_mk(source,dest)
    else:
        # so the source is a folder, 
        # now enumerate all the items in the source directory  
        # and take appropriate actions  
        for item in listdir(source):
            combine_mk(source+'/'+item,dest)



# Please Uncomment Below TWO Cells to Combine the Test & Train Data
## I did it thus I have left them commented

If you need anyhelp Sritam, just whatsapp me .. 

In [ ]:
# used to combine all the processed files for both train & test data sets 
# together one to each other in their respective files

# Combine Test Data
s="processed_data/test_corpus"
d = "processed_data/test_combine.txt"
dfile = open(d,"w+", encoding="utf-8")
dfile.close()

combine_mk(s,d)

In [ ]:
# Combine Train Data
s="processed_data/train_corpus"
d = "processed_data/train_combine.txt"
dfile = open(d,"w+", encoding="utf-8")
dfile.close()


combine_mk(s,d)

# Calculating Emission Probabilities

I saw many videos on youtube with many examples, now I think this is first step we should do. 
And if we can do this nicely, this will give us COnfidence as well :)

## What Does Emission Probability means?
Make this thing clear, so that implementation is not wrong.

Hidden states are tags 
And observations are words... 

So emission probability means, for word we calculate the chance that given tag, what is probability this word will occur 

Emission P of word w with tag t = Count of word_tag pairs with word = w and tag = t divided by count of word_tag pairs with tag equal to t no matter what word is....

### So far so good ... 

Obviously this makes things clear.

I will make a dictionary counting appearnace of every tag , this will give me count of word_tag pairs with tag t for every tag t

Or better make dictionary of dictionary
Dict with keys as tags, and second dictionary with keys as words ? how's this ... seems fine right... this will give us all the data we need to go 

Finally to save it I will pickle the struct, and will see if I need to do anything else.

In [ ]:
import pickle

In [ ]:
# the emission probabilty calculator functtion

def emission_mk():
    # obviosly source file is 
    data = open("processed_data_sentence/train_sentence_combine_.txt", "r", encoding="utf-8")
    
    print("file successfully loaded")
    # make dictonary for tags as keys
    # this will have tag as keys and values as their count
    tag_dict = {}
    
    # make dictionary for tags and keys 
    # values will be another dictionary for words
    tag_dict_dict = {}
    
    # count number of sentences
    count = 0
    # now read every line one by one of this file
    for line in data :
        
        # if line is a new line then skip this line
        if len(line) < 2 :
            continue
            
        # split line into word_tag pairs with split
        pairs = line.split(" ")
        
        
        # iterate over pairs
        for pair in pairs :
            # split the give pair 
            try :
                [word,tag] = pair.split("_")
            except : 
                print(pair)
                continue
                    
            [word,tag] = pair.split("_")
            # increase frequency for this tag in tag dictinary
            tag_dict[tag] = tag_dict.get(tag,0) + 1
            
            # obtain the dictionary with given tag as key
            # in case the tag is not there, then the default value must be returned : empty dictionary
            temp_word_dict = tag_dict_dict.get(tag, {}) 
            # now increase the count in the word dictinary for corresponding word
            temp_word_dict[word] = temp_word_dict.get(word,0) + 1
            # now assign this dictionary to the system
            tag_dict_dict[tag] = temp_word_dict
            
            # all dictionaries filled properly
        # pairs in sentence processed
        count = count +1
        print("sentence number :", count, " processed")
    # all the lines are now processed
    
    # now all dictionaries are properly filled with values
    
    #pickle the first dictionary
    tag_dict_picklefile = open("pickle/tagscount","wb+")
    pickle.dump(tag_dict, tag_dict_picklefile)
    tag_dict_picklefile.close()
    
    # count emission probabilties
    # to do so we divide the word counts given tag with the count of the tag
    # iterate over all the tags in the system
    for tag in tag_dict :
        
        if tag_dict[tag] == 0:
            continue
        # for every word in the corresponding dictionary of the word
        for word in tag_dict_dict[tag]:
            tag_dict_dict[tag][word] = (1.0*tag_dict_dict[tag][word]) / tag_dict[tag]
        # all words for this tag settled
    # all tags settled
    
    #pickle the second dictionary
    emission_pickle = open ("pickle/emission", "wb+")
    pickle.dump(tag_dict_dict,emission_pickle)
    emission_pickle.close()
    
    # function complete        
    
        
        
        
        

#  Please Uncomment Below Code to make the emission probability pickle object

In [ ]:
# Run the above function for generating the files
emission_mk()

# Transission Probabilties

what are they, first of all 

Basically they predict that after the apearance of certain tag t1 (say), what is the probability that next tag will be t2(saY). 


T2 may or may not be equal to T1.

## Next Obvious Question, How to get these values

I think, I will iterate sentence wise, with each sentence begining with a special tag, called start, we will store the count of sentences (something around 4lac 32 thousands or 4lac 83 thousands. I don't remember exact values.

I got this count during emission probabilty.
Unfortunately I didnot saved this data.

Never mind.
Let's get back to task!

So Transission P (t2) with t1 = Num of times t2 occurs after occurance of t1 / num of times t1 occurs in the data set. .... 

the denominator is simply the tag count pickle, which is to be unloaded. :) 

for numerator I need to store the number of times, t2 occurs after t1 .

Example : Mohit (Noun) is(verb) a(adj) Boy(noun). So we will make a new tag, "START"
New Sentence will be : ""(START) Mohit (Noun) is(verb) a(adj) Boy(noun). 

let's start

In [ ]:
#function to find transmission probabilities
def transmission_mk():
    
    # open the file
    data = open("processed_data/train_combine.txt", "r", encoding="utf-8")
    print("file successfully loaded")
    
    # create dictionary for storing the tag pairs
    # structure tag_dict[t1][t2] = number of times t2 appears after the occurence of t1
    tag_dict = {}
    
    # counter 
    count = 0
    
    #iterate over every sentence in the train_combine.txt processed data file
    for line in data : 
        
        # strip end of line characters
        line = line.strip()
        
        # skip empty line
        if len(line) < 2 :
            continue
        
        # find pairs in the line
        pairs = line.split()
        
        # initialize the start tag value
        prev_tag = "START"
        
        
        # iterate over every pair
        for pair in pairs : 
            # split word tag pair
            try :
                [word,cur_tag] = pair.split("_")
            except : 
                print(pair)
                continue
                    
            [word,cur_tag] = pair.split("_")
            
            # obtain the dictionary for previous tag
            prev_tag_dict = tag_dict.get(prev_tag, {})
            
            # update the value of this tag, in the dictionary of previous tag
            prev_tag_dict[cur_tag] = prev_tag_dict.get(cur_tag,0)+1
            
            # updtate the complete main dictionary with new values
            tag_dict[prev_tag] = prev_tag_dict
            prev_tag=cur_tag
        # sentence iterated
        # increase count
        count = count +1
        
        # print statement for motivation purposes
        if count == 10000:
            print(count, " sentences processed")
    # whole file done
    print("total number of sentences in the text " , count)
    
    # now unpickle the tag file
    tags_count_file = open("pickle/tagscount","rb")
    tags_count_dict = pickle.load(tags_count_file)
    
    # for this run. add aditional value
    tags_count_dict["START"] = count
    
    # iterate over every tag in the list
    for t1 in tag_dict :
        
        # now for every tag in the sub dictionary updtae the value
        for t2 in tag_dict[t1] :
            # probability is equal to number of times t2 occurs after t1
            # divided by the number of times t1 occurs in the whole train data
            # we multiply by 1.0 to get a double value
            tag_dict[t1][t2] = (1.0*tag_dict[t1][t2])/ tags_count_dict[t1]
    
    # now pickle this dictionary
    transmission_pickle = open ("pickle/transmission", "wb+")
    pickle.dump(tag_dict,transmission_pickle)
    transmission_pickle.close()
    
    print("Function completed succesfully, all glories to HDGACBVSSP, pickle saved for transmission prob")
            
        
        

# Please UNcomment below block to generate transmission probabilities

In [ ]:
# get transmission probabilities
transmission_mk()
import numpy as np

In [48]:
#viterbi algorithm implemention
#observed states are words
#hidden states are tags

def viterbi():
    
    file = open("processed_data_sentence/test_sentence_combine.txt", "r", encoding= "utf-8")
    
    pickled_emission=open("pickle/emission","rb")
    emission_dict=pickle.load(pickled_emission)
    pickled_emission.close()
    
    pickled_transition=open("pickle/transmission","rb")
    transition_dict=pickle.load(pickled_transition)
    pickled_transition.close()
    
    pickled_tags=open("pickle/tagscount","rb")
    tag_dict=pickle.load(pickled_tags)
    pickled_tags.close()
    
    tag_list=[*tag_dict]
    
    total_tag=tag_list
    predicted=0
    actual=0
    confusion_dict={}
    cnt=0
    for sentence in file:
        
        #initialising
        #extracting tag list

        #storing the the total probability
        final_value={}
        sentence=sentence.strip()
        if len(sentence) <2:
            continue
        pairs = sentence.split()
        wordlis = []
        taglis = []


        for pair in pairs :
            try :
                [word,trag] = pair.split("_")
            except :     
                print(pair)
                continue
            [word,tag] = pair.split("_")
            wordlis.append(word)
            taglis.append(tag)
            

        for tag in tag_list:
            
            a=emission_dict[tag].get(word,0.00000001)
            b=transition_dict["START"].get(tag,0.00000001)

            final_value[tag]=a*b
            
        
        final_value_tmp={}
        final_value_tmp=final_value
        result =[]
        for word in wordlis[1:]:
            
            final_value=final_value_tmp
            result.append(max(final_value, key=final_value.get))
            
            final_value_tmp={}
            
            for tag_cur in tag_list:
                tmp=[]
                for tag_prev in tag_list:
    #                 a=emission_dict[tag_cur].get(word,0.00000001)
    #                 b=transition_dict[tag_prev].get(tag_cur,0.00000001)

                    if word in emission_dict[tag_cur]:
                        a=emission_dict[tag_cur][word]
                    else:
                        a=0.00000001
                    if tag_prev in transition_dict and tag_cur in transition_dict[tag_prev]:
                        b=transition_dict[tag_prev][tag_cur]
                    else:
                        b=0.00000001

                    tmp.append(a*b*final_value[tag_prev])
                final_value_tmp[tag_cur]=max(tmp)
        result.append(max(final_value, key=final_value.get))
        
        cnt =cnt + 1
        if cnt%10000==0:
            print(cnt, " sentences processed")
            break
        
#         print(taglis)
#         print(result)
        actual +=len(result)
        predicted +=len(result)
        for i in range(len(result)):
            
            if taglis[i] not in total_tag:
                total_tag.append(taglis[i])
                
            tmp_dict=confusion_dict.get(taglis[i],{})
            tmp_dict[result[i]]=tmp_dict.get(result[i],0)+1
            confusion_dict[taglis[i]]=tmp_dict
            
            if taglis[i] != result[i]:
                predicted =predicted-1
                
    print(predicted/actual)
    
    mat=[[0 for i in range(len(total_tag))] for j in range(len(total_tag))]

    for i in range(len(total_tag)):
        for j in range(len(total_tag)):
            tmp_dict=confusion_dict.get(total_tag[i],{})
            mat[i][j]=tmp_dict.get(total_tag[j],mat[i][j])

        
    np.set_printoptions(threshold=np.inf ,linewidth=np.inf)
    # define data
    TAG = np.asarray(total_tag)
    # save to csv file
    np.savetxt('total_tags.csv', TAG.astype(str), fmt='%s', delimiter=',')

    np.set_printoptions(threshold=np.inf ,linewidth=np.inf)
    # define data
    data = np.asarray(mat)
    # save to csv file
    np.savetxt('confusion_matrix.csv', data.astype(int), fmt='%d', delimiter=',')
    

In [49]:
viterbi()

10000  sentences processed
0.8077640944526042


In [ ]:
#confusion matrix



